# Docker Compose

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/ok-transformer/blob/master/docs/nb/dk/01-compose.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/ok-transformer?style=social)](https://github.com/particle1331/ok-transformer)

---

## Introduction

Multi-container applications require configuring setup and tear down of run and builds, volumes, as well as networking between multiple services. This can be tedious to do using Docker CLI commands especially during development. **Docker Compose** allows us to collect all of these in an intuitive and human-readable format using a YAML file. This also automatically takes care of networking between containers as well as logging and status monitors for the whole ensemble. The idea is that multiple containers which function as a single entity should be managed as a single entity. To demonstrate a use case, we create two services consisting of a simple web application and a [redis](https://redis.io/) server as in-memory database.

## FastAPI app

First we create a simple FastAPI web app. This installs pipenv on system python which is generally not recommended, but is fine here because of container isolation. This will demonstrate how to run a web application using Docker. This app defines a single endpoint: 

```python
@app.get("/")
def hello():
    return {"message": "hello, world!"}
```

### Build

Setting `/usr/app` as the **working directory**. This is where all subsequent build commands will be executed. The copy command can be confusing. See the following figure. Note that we structure the application so that source files are separated in a `/src` folder while the other project files are in the root folder.

In [1]:
!cat simple-web/Dockerfile

FROM python:3.9.15-slim

RUN pip install -U pip
RUN pip install pipenv

WORKDIR /usr/app

COPY Pipfile Pipfile.lock ./
RUN pipenv install --system --deploy

COPY ./src/ ./src/

CMD uvicorn src.main:app --port 8080 --host 0.0.0.0


```{figure} diagrams/01-copy.svg
---
width: 330px
name: copy
---
Copies files from a local path to a path inside the container. The trailing `/` is important.
```

Building:

In [2]:
!docker build simple-web -t okt/simple-web --quiet
!docker run --rm -d okt/simple-web:latest

sha256:b24449de78e7f20e0956c0fcec900608e09778c01d00c03b6f13a5eb22a1e447
d984e5723643808c34bbc5e5f9be41be88ffba8cd5a231a1b3841b1a7c5d2c7c


Checking if the file structure is as intended:

In [3]:
!tree simple-web

simple-web
├── Dockerfile
├── Pipfile
├── Pipfile.lock
└── src
    └── main.py

2 directories, 4 files


In [4]:
!docker exec d984e57236 apt update > /dev/null
!docker exec d984e57236 apt install tree > /dev/null
!docker exec d984e57236 tree

.
├── Pipfile
├── Pipfile.lock
└── src
    ├── __pycache__
    │   └── main.cpython-39.pyc
    └── main.py

2 directories, 4 files


### Setting up ports

In [6]:
!docker logs d984e57236

INFO:     Started server process [7]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


Note that this port is not accessible to us. To access this, we have to publish ports:

In [7]:
!http :8080/


http: LogLevel.ERROR: ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10459dfd0>: Failed to establish a new connection: [Errno 61] Connection refused')) while doing a GET request to URL: http://localhost:8080/




<br>

```{figure} diagrams/01-port-mapping.svg
---
width: 800px
name: port
---
Route incoming requests to port 3000 on local host to port 8080 inside the container.
```

In [8]:
!docker run --rm -d -p 3000:8080 okt/simple-web:latest

c628f312f4bed1fa9f4ad12be9651f9d0ddca91fbe6d0739ef406b73580903c5


In [9]:
!http GET :3000/

HTTP/1.1 200 OK
content-length: 27
content-type: application/json
date: Sun, 26 Mar 2023 23:48:38 GMT
server: uvicorn

{
    "message": "hello, world!"
}




In [10]:
!docker stop c628f312f

c628f312f


## Multiple services

Here we will build a web server which extends a bit the above simple web app to display the number of visits of a page. This responds to HTTP requests and generates HTML to show inside a browser. To actually store the number of times a page is visited, we will make use of a little redis server. Note that we can install the redis server inside the web server itself. But this setup does not scale well, e.g. to having multiple servers that depend on the same number of visits value. Also, we lose all our data when the web server fails. So this is a more robust setup.

### Web server

Our web server will implement the following endpoint. Notice that the host is `redis-server` instead of a URL. This is one magic of using `docker-compose` where we only need to specify the name of our service. This is discussed in the next section.

```python
app = FastAPI()
r = redis.Redis(host='redis-server', port=6379)
if not r.exists('visits'):
    r.set('visits', 0)


@app.get("/", response_class=HTMLResponse)
def home():
    visits = int(r.get('visits')) + 1
    r.set('visits', visits)
    return f"Number of visits: {visits}"
```

The Dockerfile for this service is almost the same as our simple web app above. Note that we deploy using [gunicorn with uvicorn workers](https://fastapi.tiangolo.com/deployment/server-workers/) as recommended in the docs. 

In [11]:
!cat visits/app/Dockerfile

FROM python:3.9.15-slim

RUN pip install -U pip
RUN pip install pipenv

WORKDIR /usr/app

COPY Pipfile Pipfile.lock ./
RUN pipenv install --system --deploy

COPY ./src/ ./src/

CMD gunicorn src.main:app --workers 1 --worker-class uvicorn.workers.UvicornWorker --bind 0.0.0.0:8081


### Redis DB

For our redis server, we simply pull the image from Docker Hub.

In [12]:
!docker run --rm -d redis

ac51d308700ff237f778ef3830b11f1ce17a04a04b928501afe86a538584b492


In [13]:
!docker logs ac51d30870

1:C 26 Mar 2023 23:49:46.963 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
1:C 26 Mar 2023 23:49:46.963 # Redis version=7.0.10, bits=64, commit=00000000, modified=0, pid=1, just started
1:C 26 Mar 2023 23:49:46.963 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
1:M 26 Mar 2023 23:49:46.963 * monotonic clock: POSIX clock_gettime
1:M 26 Mar 2023 23:49:46.965 * Running mode=standalone, port=6379.
1:M 26 Mar 2023 23:49:46.965 # Server initialized
1:M 26 Mar 2023 23:49:46.967 * Ready to accept connections


Setting visit count initially to zero:

In [14]:
!docker exec ac51d30870 redis-cli set visits 0

OK


In [15]:
!docker exec ac51d30870 redis-cli get visits

0


## Docker Compose

As mentioned, to manage our two services easily, we use `docker-compose` version 3. Here services means our two containers which perform two specific purposes. To configure, we have to specify a `docker-compose.yml` file. 

In [16]:
!cat visits/docker-compose.yml

version: '3'
services:
  redis-server:
    image: redis
  web-server:
    restart: on-failure
    build: app
    ports:
      - 3001:8081


As mentioned above, the redis server built from the `redis` image (pulled if not locally available) will be hosted on the host `'redis-server'` which will be accessible from the `web-server` app. The build path `app/` specifies the location of the Dockerfile for this service relative to the YAML file. Finally, we have to publish the container port 8081 to 3001 on our local machine which runs the Docker server. The ports is a list as indicated by `-`, so we can publish as many ports as we want.

In [17]:
!tree visits

visits
├── app
│   ├── Dockerfile
│   ├── Pipfile
│   ├── Pipfile.lock
│   └── src
│       └── main.py
└── docker-compose.yml

3 directories, 5 files


If we have another service that we manually build (i.e. not simply pull), it will be in the same level as `app/`. The `docker-compose.yml` file is on the same level as the service directories. Here we only have one.

### Compose up

The `--build` flag rebuilds the images. This is not really necessary for a first build since the images are automatically built with `docker-compose up` if the images are not available. Running the entire ensemble:

In [18]:
import os; os.chdir('visits')
!docker-compose up -d --build

[+] Building 0.0s (0/0)                                                         
[+] Building 0.0s (0/0)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 32B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.9.15-slim      0.0s
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 32B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B     

It is important to note the other services which has no change will not be rebuilt. These services can persist data and state while rebuilt services have reset state. To view running processes, we use:

In [19]:
!docker-compose ps

NAME                    COMMAND                  SERVICE             STATUS              PORTS
visits-redis-server-1   "docker-entrypoint.s…"   redis-server        running             6379/tcp
visits-web-server-1     "/bin/sh -c 'gunicor…"   web-server          running             0.0.0.0:3001->8081/tcp


Due to container isolation, a fresh redis DB is instantiated when the services are up:

In [20]:
!http :3001

HTTP/1.1 200 OK
content-length: 19
content-type: text/html; charset=utf-8
date: Sun, 26 Mar 2023 23:50:49 GMT
server: uvicorn

Number of visits: 1




In [21]:
!http :3001

HTTP/1.1 200 OK
content-length: 19
content-type: text/html; charset=utf-8
date: Sun, 26 Mar 2023 23:50:50 GMT
server: uvicorn

Number of visits: 2




<br>

```{figure} diagrams/01-containers.png
---
name: containers
---
Running composed services from Docker Desktop.
```


```{figure} diagrams/01-compose-monitoring.png
---
name: compose-monitoring
---
Monitoring running services. Note that a terminal can also be opened from inside the container.
```

### Restart policies

Note that a web service typically crashes for a variety of reasons. Oftentimes this can happen rarely so that we want to restart one particular service immediately after it crashes. In the `docker-compose.yml` we specified `on-failure` as the restart policy. This means the service is restarted every time it fails with nonzero exit status, this is different from being stopped which has exit status `0`.

<br>

```{figure} diagrams/01-restart-policies.png
---
width: 650px
name: restart-policies
---
[Restart policies](https://docs.docker.com/config/containers/start-containers-automatically/#use-a-restart-policy) for containers in Docker.
```

To demonstrate, we defined an endpoint which results in a failure as described:

```python
@app.get("/kill")
async def kill_uvicorn():
    parent_pid = os.getppid()
    os.kill(parent_pid, 9)
```

In [24]:
!http :3001/kill

HTTP/1.1 200 OK
content-length: 4
content-type: application/json
date: Sun, 26 Mar 2023 23:51:09 GMT
server: uvicorn

null




In [25]:
!docker-compose ps

NAME                    COMMAND                  SERVICE             STATUS              PORTS
visits-redis-server-1   "docker-entrypoint.s…"   redis-server        running             6379/tcp
visits-web-server-1     "/bin/sh -c 'gunicor…"   web-server          restarting          


Note restarting status. After the service starts, the previous number of visits is persisted:

In [26]:
!http :3001/

HTTP/1.1 200 OK
content-length: 19
content-type: text/html; charset=utf-8
date: Sun, 26 Mar 2023 23:51:15 GMT
server: uvicorn

Number of visits: 3




<br>

```{figure} diagrams/01-restart-failure.png
---
width: 650px
name: 01-restart-failure
---
Logs when the web service exits with code 173. Notice it immediately restarts.
```

Stopping does not start the service. For this we need the other restart policies.

In [27]:
!docker-compose ps -q web-server

581285d6e75af6e8eaa84b050c7ea4c7cf7d83e8c220db5df4cdf77c87f1fe57


In [28]:
!docker stop 581285d6e75

581285d6e75


In [29]:
!docker-compose ps

NAME                    COMMAND                  SERVICE             STATUS              PORTS
visits-redis-server-1   "docker-entrypoint.s…"   redis-server        running             6379/tcp
visits-web-server-1     "/bin/sh -c 'gunicor…"   web-server          exited (137)        


### Compose down

Stopping and teardown of all the services:

In [30]:
!docker-compose down

[+] Running 1/0
 ⠿ Container visits-web-server-1    Rem...                                 0.0s
 ⠋ Container visits-redis-server-1  S...                                   0.1s
[+] Running 1/2
 ⠿ Container visits-web-server-1    Rem...                                 0.0s
 ⠙ Container visits-redis-server-1  S...                                   0.2s
[+] Running 3/2
 ⠿ Container visits-web-server-1    Rem...                                 0.0s
 ⠿ Container visits-redis-server-1  R...                                   0.2s
 ⠿ Network visits_default           Removed                                0.0s
